# nlp topic modeling

In [ ]:
import pandas as pd

df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.1})

In [ ]:
df = df.iloc[0, :]
df

In [ ]:
df["text"] = [df["news"][i]["ExtractedText"] for i in range(len(df["news"]))]

In [ ]:


df = pd.DataFrame(df)
df = df.transpose()
df

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

nltk.download('stopwords')

In [ ]:
df_new = pd.DataFrame({})
df_new

In [ ]:
texts = df['text'][0]

stop_words = stopwords.words('portuguese')

def preprocess(text):
    # Lowercase, remove non-alphabetic characters
    text = text.lower()
    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    # Remove stopwords
    words = [word for word in text.split() if word not in stop_words]
    return ' '.join(words)

df_new['cleaned_text'] = pd.Series(texts).map(preprocess)
df_new['cleaned_text'] = df_new['cleaned_text'].fillna('')

vectorizer = CountVectorizer(max_df=0.95, min_df=2)
dtm = vectorizer.fit_transform(df_new['cleaned_text'])



n_topics = 5  # Adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_model.fit(dtm)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx+1}:")
        print(", ".join([feature_names[i] for i in topic.argsort()[-no_top_words:]]))

no_top_words = 10
display_topics(lda_model, vectorizer.get_feature_names_out(), no_top_words)

In [ ]:
# Vocabulary as a list of terms
vocab = vectorizer.get_feature_names_out()

# Term frequencies (sum of word counts across all documents)
term_frequency = dtm.sum(axis=0).A1


In [ ]:
import pyLDAvis

pyLDAvis.enable_notebook()

vis_data = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_,
    doc_topic_dists=lda_model.transform(dtm),
    doc_lengths=dtm.sum(axis=1).A1,
    vocab=vocab,
    term_frequency=term_frequency
)

pyLDAvis.display(vis_data)
